# Redis Caching

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd 
import numpy as np 
from urllib.request import urlopen
import redis
import json
import pysolr
import requests

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
site = "http://awesome-hw.sdsc.edu:8983/solr/#/student-news/query"

## Queries

### (1) - Trump and Taxes

In [5]:
q1j = requests.get('http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22trump%22%20AND%20title%3A%22taxes%22')
q1j.json()

{'responseHeader': {'status': 0,
  'QTime': 0,
  'params': {'q': 'title:"trump" AND title:"taxes"'}},
 'response': {'numFound': 207,
  'start': 0,
  'docs': [{'src': 'http://www.nytimes.com/pages/todayspaper/index.html',
    'title': 'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us',
    'news': 'We don’t know if Donald Trump’s recent federal income tax returns indicate illegal activity. But even if the $750 income tax bills he paid in each of the first two years of his presidency turn out to have been calculated in compliance with the U.S. Internal Revenue Code, he has enjoyed the benefits of the society we all contribute to build, then skipped out on his share of the bill — a sort of dine-and-dash on his fellow Americans.\n\nWhat many Americans may not realize is how common this is among the country’s top earners, and how much it costs the rest of us. Sixteen years ago, the former I.R.S. commissioner Charles Rossotti estimated that so many wealthy Americans don’t pay their

In [6]:
q1 = 'http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22trump%22%20AND%20title%3A%22taxes%22'
q1d = requests.get(q1).json()
df1 = pd.json_normalize(q1d['response']['docs'])
# df1.shape[0]
df1[:1]

,src,title,news,publishdate,id,_version_
0,http://www.nytimes.com/pages/todayspaper/index.html,Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us,"We don’t know if Donald Trump’s recent federal income tax returns indicate illegal activity. But even if the $750 income tax bills he paid in each of the first two years of his presidency turn out to have been calculated in compliance with the U.S. Internal Revenue Code, he has enjoyed the benefits of the society we all contribute to build, then skipped out on his share of the bill — a sort of dine-and-dash on his fellow Americans.\n\nWhat many Americans may not realize is how common this is among the country’s top earners, and how much it costs the rest of us. Sixteen years ago, the former I.R.S. commissioner Charles Rossotti estimated that so many wealthy Americans don’t pay their fair share of income taxes that it effectively imposes a 15 percent surtax on the rest of us — that is, on the vast majority of taxpayers who can’t afford the expensive professional advice on which successful tax-avoidance strategies depend.\n\nHaving spent the past 13 years studying and writing about these elite professionals who help the ultrarich reduce their tax bills as close to zero as possible, I understand how the game of “creative compliance” is played: They adhere to the letter of the law while violating its spirit. The tax avoidance professionals say, as Mossack Fonseca did after the Panama Papers broke, we complied with the law, so what’s the problem?\n\nMany Americans seem to accept this argument. Some even greet news of tax avoidance by the ultrarich with admiration and envy, instead of the anger they would show to someone who skipped out on a group dinner and left them with the bill. Rather than condemning those who take advantage of their society, many Americans say instead: “Good for them! I’d do it myself if I could.” Those attitudes are reflected in our politics and have contributed to our handing over the most powerful elected office in the world to a man who bragged about not paying a dime of federal income tax for years.",2020-10-01T00:00:00Z,c7168dcc-75e3-43cb-8460-30481fd336b4,1686103765257551872


### (2) - Millions and COVID 

In [7]:
q2j = requests.get("http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22millions%22%20AND%20title%3A%22covid%22%20AND%20title%3A%22deaths%22")
q2j.json()

{'responseHeader': {'status': 0,
  'QTime': 0,
  'params': {'q': 'title:"millions" AND title:"covid" AND title:"deaths"'}},
 'response': {'numFound': 34,
  'start': 0,
  'docs': [{'src': 'https://www.marketwatch.com/',
    'title': 'Coronavirus tally: Global cases of COVID-19 40.4 million, 1.1 million deaths and U.S. death toll tops 220,000',
    'news': 'The global tally for confirmed cases of the coronavirus that causes COVID-19 climbed above 40.4 million on Tuesday, according to data aggregated by Johns Hopkins University, while the death toll rose above 1.1 million. The U.S. has the highest case total at 8.2 million and deaths at 220,133. President Donald Trump on Monday called Dr. Anthony Fauci, head of the National Institute for Allergies and Infectious Diseases and considered the nation`s leading infectious-disease expert, a "disaster" on a campaign call and on Twitter and said, "People are tired of hearing Fauci and all these idiots." Brazil has the second highest death toll at

In [8]:
q2 = 'http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22millions%22%20AND%20title%3A%22covid%22%20AND%20title%3A%22deaths%22'
q2d = requests.get(q2).json()
df2 = pd.json_normalize(q2d['response']['docs'])
# df2.shape[0]
df2[:1]

,src,title,news,publishdate,id,_version_
0,https://www.marketwatch.com/,"Coronavirus tally: Global cases of COVID-19 40.4 million, 1.1 million deaths and U.S. death toll tops 220,000","The global tally for confirmed cases of the coronavirus that causes COVID-19 climbed above 40.4 million on Tuesday, according to data aggregated by Johns Hopkins University, while the death toll rose above 1.1 million. The U.S. has the highest case total at 8.2 million and deaths at 220,133. President Donald Trump on Monday called Dr. Anthony Fauci, head of the National Institute for Allergies and Infectious Diseases and considered the nation`s leading infectious-disease expert, a ""disaster"" on a campaign call and on Twitter and said, ""People are tired of hearing Fauci and all these idiots."" Brazil has the second highest death toll at 154,176 and is third by cases at 5.3 million. India is second in cases with 7.6 million, and third in deaths at 15,197. Mexico has the fourth highest death toll at 86,338 and ninth highest case tally at 854,926. The U.K has 43,816 deaths, the highest in Europe and fifth highest in the world, and 744,122 cases.",2020-10-20T00:00:00Z,cccc6ef3-a147-41e3-bc1b-de099daa1942,1686104086943891457


### (3) - Biden and Trump on Election Day

In [9]:
q3j = requests.get("http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22Biden%22%20AND%20title%3A%22Trump%22%20AND%20publishdate%3A%222020-11-03T00%3A00%3A00Z%22")
q3j.json()

{'responseHeader': {'status': 0,
  'QTime': 0,
  'params': {'q': 'title:"Biden" AND title:"Trump" AND publishdate:"2020-11-03T00:00:00Z"'}},
 'response': {'numFound': 234,
  'start': 0,
  'docs': [{'src': 'https://www.deseret.com/',
    'title': 'TX: Trump 50% Biden 46%',
    'news': 'In Texas, President Trump is trying to make it 11 straight victories for Republicans in the Lone Star State. The latest PoliticalIQ poll shows Trump with the support of 50% of Likely Texas Voters and former Vice President Biden with the support of 46%. President Trump’s lead is just within the margin of error of 3.5%. The poll, conducted by Scott Rasmussen, shows 2% voting for someone else and 2% not sure. The last Democrat to win Texas was Jimmy Carter in 1976.\n\nWith more than 900,000 confirmed cases, Texas is among those states leading the nation in total coronavirus cases since the start of the pandemic. This healthcare crisis adds to the difficulty in predicting voter turnout. As such, all Political

In [10]:
q3 = 'http://awesome-hw.sdsc.edu:8983/solr/student-news/select?q=title%3A%22Biden%22%20AND%20title%3A%22Trump%22%20AND%20publishdate%3A%222020-11-03T00%3A00%3A00Z%22'
q3d = requests.get(q3).json()
df3 = pd.json_normalize(q3d['response']['docs'])
# df3.shape[0]
df3[:1]

,src,title,news,publishdate,id,_version_
0,https://www.deseret.com/,TX: Trump 50% Biden 46%,"In Texas, President Trump is trying to make it 11 straight victories for Republicans in the Lone Star State. The latest PoliticalIQ poll shows Trump with the support of 50% of Likely Texas Voters and former Vice President Biden with the support of 46%. President Trump’s lead is just within the margin of error of 3.5%. The poll, conducted by Scott Rasmussen, shows 2% voting for someone else and 2% not sure. The last Democrat to win Texas was Jimmy Carter in 1976.\n\nWith more than 900,000 confirmed cases, Texas is among those states leading the nation in total coronavirus cases since the start of the pandemic. This healthcare crisis adds to the difficulty in predicting voter turnout. As such, all PoliticalIQ.com polls are released with three separate turnout models—a baseline projection, a Strong Republican Turnout model, and a Strong Democratic Turnout model.\n\nIn the Strong Republican Turnout model, Trump stretches his lead to eight points over Biden 52% – 44%. However, in the Strong Democrat Model, Biden pulls even 48%-48%.\n\nPoliticalIQ.com will have the latest from key states up until election day, including the latest on the Senate Race in Texas tomorrow. In 2016, Trump beat Hillary Clinton in Texas by 9%. The State has 38 Electoral Votes.\n\nMETHODOLOGY\n\nThe survey of 800 Likely Texas Voters was conducted by Scott Rasmussen from October 27-28, 2020. Field work for the survey was conducted by RMG Research, Inc. Respondents were randomly selected from a list of Registered Voters and contacted via text or through a process of Random Digital Engagement. The Likely Voter sample was derived from a larger sample of Registered Voters using screening questions and other factors. Certain quotas were applied to the larger sample and lightly weighted by geography, gender, age, race, education, and political party to reasonably reflect the state’s population of Registered Voters. Other variables were reviewed to ensure that the final sample is representative of that population.\n\nForty-nine percent (49%) of the Likely Voters either identify as Republican or Lean Republican. Forty-four percent (44%) either identify as a Democrat or Lean Democrat. Seven percent (7%) do not identify with either major party.\n\nThe margin of sampling error for the full sample is +/- 3.5 percentage points.",2020-11-03T00:00:00Z,03f1d1c6-a021-4a5f-81d8-8a85e63452a3,1686104344201527296


## Implementing a Redis Cache

### Query 1

In [11]:
r1 = redis.Redis(host='my-redis', port=6379, db=0)

In [12]:
dict1 = dict(zip(df1['title'], df1['publishdate']))
dict1

{'Opinion | Trump’s Tax Avoidance Is a Tax on the Rest of Us': '2020-10-01T00:00:00Z',
 'Brooke Harrington: Trump’s tax avoidance is a tax on the rest of us': '2020-10-03T00:00:00Z',
 'Trump Tax-Return Report Fuels Democrats’ Drive to Tax the Rich': '2020-10-01T00:00:00Z',
 'Letter: Trump, Taxes and naive thinking': '2020-10-15T00:00:00Z',
 'President Trump’s Taxes with Joe Bankman': '2020-10-25T00:00:00Z'}

In [13]:
for key in dict1.keys():
    for val in dict1[key]:
        r1.append(key, val)

In [14]:
pipe = r1.pipeline()
pipe.get('Brooke Harrington: Trump’s tax avoidance is a tax on the rest of us')
pipe.execute()

[b'2020-10-03T00:00:00Z']

### Query 2

In [15]:
r2 = redis.Redis(host='my-redis', port=6379, db=0)

In [16]:
dict2 = dict(zip(df2['title'], df2['publishdate']))
dict2

{'Coronavirus tally: Global cases of COVID-19 40.4 million, 1.1 million deaths and U.S. death toll tops 220,000': '2020-10-20T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 40.8 million, 1.13 million deaths and U.S. death toll tops 221,000': '2020-10-21T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 41.28 million, 1.13 million deaths and U.S. death toll tops 222,000': '2020-10-22T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 34.7 million, 1.03 million deaths and U.S. death toll nears 209,000': '2020-10-03T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 35.5 million, 1.04 million deaths and U.S. death toll tops 210,000': '2020-10-06T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 38.2 million, 1.09 million deaths and U.S. death tolls nears 216,000': '2020-10-14T00:00:00Z',
 'Coronavirus tally: Global cases of COVID-19 top 34.3 million, 1.02 million deaths and U.S. death toll tops 207,000': '2020-10-02T00:00:00Z',
 'Coronavirus tally:

In [17]:
for key in dict2.keys():
    for val in dict2[key]:
        r2.append(key, val)

In [18]:
pipe = r2.pipeline()
pipe.get('Coronavirus tally: Global cases of COVID-19 40.4 million, 1.1 million deaths and U.S. death toll tops 220,000')
pipe.execute()


[b'2020-10-20T00:00:00Z']

### Query 3

In [19]:
r3 = redis.Redis(host='my-redis', port=6379, db=0)

In [20]:
dict3 = dict(zip(df3['title'], df3['publishdate']))
dict3

{'TX: Trump 50% Biden 46%': '2020-11-03T00:00:00Z',
 'MT: Trump 50% Biden 46%': '2020-11-03T00:00:00Z',
 'WI: Biden 50% Trump 44%': '2020-11-03T00:00:00Z',
 'PA: Biden 49% Trump 43%': '2020-11-03T00:00:00Z',
 'In 2020 finale, Trump combative, Biden on offense': '2020-11-03T00:00:00Z',
 'Battleground Michigan: Biden 51% Trump 44%': '2020-11-03T00:00:00Z',
 'A nation on edge, Biden and Trump race to the finish': '2020-11-03T00:00:00Z'}

In [21]:
for key in dict3.keys():
    for val in dict3[key]:
        r3.append(key, val)

In [22]:
pipe = r3.pipeline()
pipe.get('TX: Trump 50% Biden 46%')
pipe.execute()

[b'2020-11-03T00:00:00Z']